# 1. **Define dependencies and constrains**

In order to download tweet from Twitter, first one must create an account and apply for **developer priviledges**. The application will grant the developer basic access the the [Twitter API](https://developer.twitter.com/en/docs/twitter-api) which are not enough because it only allows the download of tweet of the last 7 days. Therefore, I've applied to the [Premium plan](https://developer.twitter.com/en/support/twitter-api/premium) which allows the download of 25k of tweets per month along with the use _full archive_ and the _30 days_ search API but with limited amout of request per month.

In [92]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
COLLAB_DIR = "/content/"

# File with Twitter project credentials
CREDENTIALS = COLLAB_DIR+'credentials.yaml'
CREDENTIALS_KEY = 'search_tweets_30_day_dev'

# csv file where tweet downloaded will be saved
DATASET = COLLAB_DIR+'dataset.csv'
DATASET_ANNOTATED = COLLAB_DIR+'dataset_annotated.csv'
SENTIPOLIC = COLLAB_DIR+'sentipolic.csv'

In [93]:
!python --version

Python 3.7.13


## Libraries

In [94]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
!apt-get install libenchant1c2a
!pip install pyenchant
!apt-get install hunspell-it

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant1c2a is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
^C


In [96]:
!apt install openjdk-8-jdk-headless -qq

import os
os.environ["JAVA_HOME"] = JAVA_HOME

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [97]:
!java -version

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [98]:
!pip install spark-nlp==3.4.4 pyspark==3.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
!pip install sparktorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ekphrasis -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## PySpark configurations

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import sparknlp

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.4")\
    .getOrCreate()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

In [ ]:
sc = spark.sparkContext
type(sc)

## Download Files from GitHub

In [ ]:
!wget https://github.com/deborahdore/italian-sarcastic-tweet-classification/raw/main/dataset/dataset.csv
!wget https://github.com/deborahdore/italian-sarcastic-tweet-classification/raw/main/dataset/other/sentipolic.csv
!wget https://raw.githubusercontent.com/deborahdore/italian-sarcastic-tweet-classification/main/credentials/credentials.yaml
!wget https://raw.githubusercontent.com/deborahdore/italian-sarcastic-tweet-classification/main/dataset/dataset_annotated.csv

In [ ]:
# italian dictionary for lemmatization
!wget https://raw.githubusercontent.com/michmech/lemmatization-lists/master/lemmatization-it.txt

# 2. **Retrieve Tweet**


> Following, some code cell will be annotated with *%% script false* in order to avoid their execution. Those cell concern the download of the tweets from Twitter. Even if this may not sound dangerous, I've finished the request at my disposal. Therefore, calling the Twitter API will produce an error. Also, please don't run them otherwise the output of the cell will be lost.



In [ ]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import DataFrame

In [ ]:
# useful imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import requests
import json
import yaml
import csv
import pdb
import pandas as pd

- First we must retrieve and validate the credentials that we will need to access the Twitter API. I've store the bearer token in a yaml file: *credentials.yaml*





In [ ]:
def handle_credentials(credentials, key):
  with open(credentials, "r") as stream:
    try:
        credentials = yaml.safe_load(stream)
        return credentials[key]
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
credentials = handle_credentials(CREDENTIALS, CREDENTIALS_KEY)
endpoint = credentials['endpoint'] # we will use this endpoint to search for the tweet
print(endpoint)

- Second we must create the header for the request

In [ ]:
def handle_headers(credentials:dict):
  headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {credentials["bearer_token"]}'
  }
  return headers

In [ ]:
headers = handle_headers(credentials)
headers

- Another parameter of the request is the query. The query determines which tweet will be returned in the response. In our case, we have 2 types of queries: the one that searches for sarcastic tweets and the one that returns non-sarcastic tweets

For the query about sarcastic tweet I've chosen some keyword that, in my opion, are used to express sarcasm and/or irony (sarcasm is a sub-type of irony):


1. sarcasmo (with or without #)
2. ironia (with or without #)
3. "*ridiamo per non piangere*"
4. #coincidenze (.. io non credo) is mostly used to express sarcasm
5. "*qualquadra non cosa*"

Many studies also suggest that sarcasm can be found in tweet related to politics. Therefore, these seems very good starting point:
1. monti, draghi, berlusconi (known italian prime minister)
2. governo
3. premier


For non-sarcastic tweet, I've excluded all the possibile word that may refer to sarcasm.

The list of operator used can be found in the [Twitter API documentation](https://developer.twitter.com/en/docs/twitter-api/enterprise/rules-and-filtering/operators-by-product).

In [ ]:
sarcasm_query = '(#sarcasmo OR sarcasmo OR #ironia OR ironia OR "ridiamo per non piangere" \
                  OR #coincidenze OR "qualquadra non cosa" OR draghi OR monti OR berlusconi \
                  OR governo OR premier) lang:it -has:media'

non_sarcasm_query = '-"ridiamo per non piangere" -sarcasmo -ironia -"qualquadra non cosa" lang:it -has:media'

- Now we can define the function that will handle the request and the dataframe where tweet will be stored.


> Other parameters that we need in order to process the request are:
- *max_result_per_page* : the maximum number of tweets per call 
- *next_token* : a token that if passed to the request will return the next page of results
- I've defined a parameter *max_num_of_request* that will stop the call once that we've reached the desidered amount of calls. This must be done because the request at our disposal are not illimited. So we must be careful to the number of the request that we do




In [ ]:
def handle_request(endpoint, headers, query, max_result_per_page, next_token = None):
  
  if next_token is not None:
    payload = json.dumps({
      "maxResults": max_result_per_page,
      "query": query,
      "next": next_token
    })
  else:
    payload = json.dumps({
      "maxResults": max_result_per_page,
      "query": query,
    })
  
  response = requests.post(endpoint, headers=headers, data=payload)

  return response.text

In [ ]:
def extract_tweet(response, label):
  tweets = []
  json_response = json.loads(response)
  
  if 'results' in response:
    results = json_response["results"]

    for tweet in results:
      # is tweet a retweet?
      if 'retweeted_status' in tweet:
        if tweet['retweeted_status']['truncated']:
          text = tweet['retweeted_status']['extended_tweet']['full_text']
        else:
          text = tweet['retweeted_status']['text']
      else:
        if tweet['truncated']:
          text = tweet['extended_tweet']['full_text']
        else:
          text = tweet['text']
        
      text = text.replace('"', "'")
      data = Tweet(tweet["id"], f"{text}", label)
      
      tweets.append(data)

  else:
    print("Request went wrong")
    print(response)

  return tweets

In [ ]:
def download_tweet(endpoint, 
                   headers, 
                   query, 
                   label,
                   max_result_per_page,
                   tweet_list,
                   next_token = None, 
                   max_num_of_request = 20):

  if max_num_of_request <= 0:
    return tweet_list

  response = handle_request(endpoint, headers, query, max_result_per_page, next_token)

  tweet_list.extend(extract_tweet(response, label))

  try:
      next_token = json.loads(response)['next']
  except:
      next_token = None

  if next_token is not None:
      return download_tweet(endpoint, headers, query, label, max_result_per_page,
                   tweet_list, next_token, max_num_of_request - 1)
  else:
      return tweet_list

In [ ]:
# define tweet
Tweet = Row("id", "text", "sarcastic")

In [ ]:
tweets = []

In [ ]:
%%script false

# download sarcastic tweet
tweets = download_tweet(endpoint, 
                   headers, 
                   sarcasm_query, 
                   "Yes",
                   100,
                   [],
                   next_token = None, 
                   max_num_of_request = 40)

In [ ]:
%%script false

# download non-sarcastic tweet
tweets.extend(
    download_tweet(endpoint, 
                   headers, 
                   non_sarcasm_query, 
                   "No",
                   100,
                   [],
                   next_token = None, 
                   max_num_of_request = 40))

In [ ]:
%%script false
# create DataFrame
df = spark.createDataFrame(tweets)

In [ ]:
%%script false
df.show(10, truncate=False)

In [ ]:
%%script false

# create file
if not os.path.exists(DATASET):
  os.mknod(DATASET)

# save tweets
df.toPandas().to_csv(DATASET, header=True, index=False) 

# 3. **Annotate Tweet**

In [ ]:
# python widgets
from ipywidgets import Button
import asyncio
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import HBox, Layout
import time as t

When we download tweet using an hashtag, we are not 100% sure of what we downloaded is correct. We must analyze - at least - the majority of the tweet to understand if what we have labelled is correct. There here's a little tool to help us with that.

In [ ]:
Tweet = Row("id", "text", "sarcastic")

schema = StructType([StructField("id", StringType(), True)\
                   ,StructField("text", StringType(), True)\
                   ,StructField("sarcastic", StringType(), True)])

df = spark.createDataFrame(pd.read_csv(DATASET), schema=schema)
df.show(10)

In [ ]:
def count_label(df, numeric=False):
  label_yes = 1 if numeric else "Yes"
  label_no = 0 if numeric else "No"
  return df.groupBy("sarcastic").agg(
      count(when(col("sarcastic") == label_yes, 1)),
      count(when(col("sarcastic") == label_no, 1)))

In [ ]:
# count tweet
print(f'Total number of tweet retrieved {df.count()}')

In [ ]:
# we want first to drop duplicates

print("Count before drop:")
count_label(df).show()

count_before_drop = df.count()
df = df.dropDuplicates(["text"])
print(f"Distinct count: {str(df.count())} \n")

print("Count after drop:")
count_label(df).show()

In [ ]:
print(f'dropped {count_before_drop-df.count()} columns')
print(f'total count: {df.count()}')

In [ ]:
# visually 
data = count_label(df).collect()

labels = ['sarcastic', 'non sarcastic']
colors = sns.color_palette('pastel')[0:5]

plt.pie([int(data[1][1]), int(data[0][2])], labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

In [ ]:
tweets_annotated = []

In [ ]:
def wait_for_change(widget1, widget2): 
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget1.on_click(getvalue, remove=True)
        widget2.on_click(getvalue, remove=True) 
    widget1.on_click(getvalue)
    widget2.on_click(getvalue)
    return future

async def f(df):
  df_pandas = df.toPandas()
  for index, row in df_pandas.iterrows():
    print(f'Is this tweet sarcastic? \n {row.text} \n', flush=True)

    x = await wait_for_change(sarcastic,non_sarcastic)
    
    if x == "Yes":
      print("Tagged ", row.id, "with sarcastic \n")
      data = Tweet(row.id, row.text, "Yes")
      tweets_annotated.append(data)
    else:
      print("Tagged ", row.id, "with non-sarcastic \n")
      data = Tweet(row.id, row.text, "No")      
      tweets_annotated.append(data)

    clear_output()
    display(HBox([sarcastic,non_sarcastic]))

Before going forward, we want to ask ourselves *How can know if a tweet is sarcastic or not?*

*In Harry Potter and the Half Blood Prince, there is a scene where Harry is leaving the Weasley house and Mrs. Weasley says: “Promise me you will look after yourself…stay out of trouble….” Harry responds: “I always do Mrs. Weasley. I like a quiet life, you know me.” Anyone familiar with Harry Potter knows that his life is far from quiet, and so he must not really mean what he is saying. In fact, Harry is being sarcastic.*

[source](https://kids.frontiersin.org/articles/10.3389/frym.2018.00056)

Sarcasm is the use of words that say the opposite of what you really mean, often as a joke and with a tone of voice that shows this. It is often used to mock or critize someone, express disapproval or as a defence mechanism.

For example:
> *Noi invece ce la caviamo con un grado in meno ai termosifoni d'inverno e spegnendo i condizionatori d'estate. Non è fantastico? (#Draghi è un cialtrone sesquipedale, nel caso aveste ancora qualche dubbio)*

Here we can imagine the sarcastic tone of the writer. He's obviously criticising the Italian prime minister, Mario Draghi, when, during an interview, he said that we must make sacrifices like lowering the grade of the radiator in order to cope with the possibility of not having the gas from Russia anymore. Obviously, this won't be enough. *Isn't this great?*

Sometimes it's difficult also for a human person to understand sarcasm therefore I don't expect the following dataset to be 100% free from bias.

In [ ]:
# tool used for annotation: it displays each tweet and the user has to click "Yes" 
# if the tweet was sarcastic, "No" otherwise

sarcastic=Button(description="Yes", button_style='info', layout=Layout(width='150px', height='50px'))
non_sarcastic=Button(description="No", button_style='info', layout=Layout(width='150px', height='50px'))

asyncio.create_task(f(df))
t.sleep(2)
display(HBox([sarcastic,non_sarcastic]))

In [ ]:
%%script false
print(tweets_annotated)

In [ ]:
%%script false
df_annotated = spark.createDataFrame(tweets_annotated)
df_annotated.tail(5)

In [ ]:
%%script false
if not os.path.exists(DATASET_ANNOTATED):
  os.mknod(DATASET_ANNOTATED)

# save tweets
df_annotated.toPandas().to_csv(DATASET_ANNOTATED, header=True, index=False) 

# 4. **Extend Dataset**

In [ ]:
schema = StructType([StructField("id", StringType(), True)\
                   ,StructField("text", StringType(), True)\
                   ,StructField("sarcastic", StringType(), True)])

df_annotated = spark.createDataFrame(pd.read_csv(DATASET_ANNOTATED), schema=schema)

In [ ]:
print(f"Annotated tweets: {df_annotated.count()}")

As we can see from the code below, we lost multiple *tweet*.
First of all, multiple tweets classified as sarcastic were not sarcastic. Also, I've dropped every tweet that contained only one word, that wasn't actually in italian or 
that had no sense.

In [ ]:
count_label(df_annotated).show()

However, we can integrate we some external Dataset such as: [SENTIPOLIC](http://www.di.unito.it/~tutreeb/sentipolc-evalita16/index.html) from the challenge EVALITA2016 which contains several italian tweet already classified.

In [ ]:
df_sentipolic = spark.createDataFrame(pd.read_csv(SENTIPOLIC))

In [ ]:
df_sentipolic.show(10)

In [ ]:
# we will extract only the tweets which are ironic since we have plenty non-ironic
df_sentipolic = df_sentipolic.filter(col("iro")==1)

In [ ]:
print(f"Ironic tweet retrieved: {df_sentipolic.count()}")

In [ ]:
# drop columns that we don't need
df_sentipolic = df_sentipolic.drop(*('subj', 'opos', 'oneg', 'lpos', 'lneg', 'top'))

# rename columns
df_sentipolic = df_sentipolic.withColumnRenamed("idTwitter", "id")\
                              .withColumnRenamed("iro", "sarcastic")

# change order
df_sentipolic = df_sentipolic.select("id", "text", "sarcastic")

In [ ]:
df_sentipolic.show(10)

In [ ]:
# now we want to join the two dataset. However we must use the same label for both.
# Therefore if the tweet is sarcastic, the label will be 1, 0 otherwise.


df_annotated = df_annotated.withColumn("sarcastic", 
                                         when(df_annotated.sarcastic == "Yes", 1)
                                         .when(df_annotated.sarcastic == "No", 0)                                    
                                         .otherwise(df_annotated.sarcastic))

In [ ]:
df_annotated.show()

In [ ]:
# concatenate DataFrames

df_complete = df_annotated.union(df_sentipolic)
df_complete.show(5)

In [ ]:
print(f'Now we have a total of {df_complete.count()} tweets')

In [ ]:
count_label(df_complete, numeric=True).show()

The dataset is still unbalanced, but better than before.

# 5. **Data Processing**

In [ ]:
import enchant
from enchant.checker import SpellChecker

In [ ]:
from pyspark.sql.functions import udf, col, lower, trim, regexp_replace, explode

In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

First we want to clean tweet: remove hashtag, links, emoji, whitespaces, mentions.

In [ ]:
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,
    segmenter="twitter", 
    corrector="twitter", 
    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=False,
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
)

In [ ]:
@udf
def spell_checker(text):
  checker = SpellChecker("it_IT", text)
  for err in checker:
    if len(err.suggest())>0:
      sug = err.suggest()[0]
      err.replace(sug)
  return checker.get_text()

In [ ]:
@udf
def tweet_processor(s):
  return " ".join(text_processor.pre_process_doc(s))

In [ ]:
def text_cleaning(df):
  print('1. Convert to lowercase')
  df = df.withColumn('text', lower(col('text')))

  print('2. Remove links')
  df = df.withColumn('text', regexp_replace('text', r'http\S+', ''))

  print("3. Remove unwanted symbols")
  df = df.withColumn('text', regexp_replace('text', '\n', ''))

  print("4. Remove digits")
  df = df.withColumn('text', regexp_replace('text', r'[0-9]{5,}', ''))

  #print("5. Remove punctuation")
  #df = df.withColumn('text', regexp_replace('text', '[^a-zA-Z\\s]', ''))

  '''
  When annotating the tweets, I've noticed that many of them contained spelling 
  errors. It is recommended to adjust those tweets before the model training.
  '''
  #print("6. Spell Checker")
  #df = df.withColumn('text', spell_checker(col('text')))

  print("7. Trimming")
  df = df.withColumn('text', trim(col('text')))

  print("8. Filter out extra whitespaces")
  df = df.withColumn('text', regexp_replace(col("text"), " +", " "))

  print("9. Text processor")
  '''
  it allows to normalize the text replacing each term with a fixed tuple 
  < [entitytype] > and to enclosing hashtags with two tags 
  < hashtag > ... < /hashtag >.
  '''
  df = df.withColumn('text', tweet_processor(col('text')))

  return df

In [ ]:
broker = enchant.Broker()
broker.describe()
broker.list_languages()

In [ ]:
df_cleaned = text_cleaning(df_complete)

In [ ]:
df_cleaned.cache()

In [ ]:
df_cleaned.show(5, truncate=False)

# 6. **Feature Engineering**

In [70]:
# libraries for feature engineering
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.annotator import Tokenizer

In [71]:
print("Starting feature engineering, constructing pipeline..")

Starting feature engineering, constructing pipeline..


In [72]:
def construct_pipeline():
  document_assembler = DocumentAssembler()\
                        .setInputCol('text')\
                        .setOutputCol('document')\
                        .setCleanupMode("shrink")
  sentence_detector = SentenceDetector()\
                      .setInputCols('document')\
                      .setOutputCol('sentence')

  tokenizer = sparknlp.annotator.Tokenizer()\
                    .setInputCols(["sentence"])\
                    .setOutputCol("token")
  
  lemmatizer = Lemmatizer()\
     .setInputCols(['token'])\
     .setOutputCol('lemma')\
     .setDictionary("lemmatization-it.txt", "->", "\t")

  stopwords_cleaner = StopWordsCleaner.pretrained("stopwords_it", "it")\
     .setInputCols(['lemma'])\
     .setOutputCol('clean_lemma')

  finisher = Finisher()\
            .setInputCols("clean_lemma")\
            .setOutputCols("pipeline_result")\
            .setCleanAnnotations(False)

  return Pipeline(stages=[
                    document_assembler,
                    sentence_detector,
                    tokenizer,
                    lemmatizer,
                    stopwords_cleaner,
                    finisher])

In [73]:
pipeline = construct_pipeline()

stopwords_it download started this may take some time.
Approximate size to download 2.4 KB
[OK!]


In [74]:
%%time
nlp_pipeline = pipeline.fit(df_cleaned)
df_engineer = nlp_pipeline.transform(df_cleaned)

CPU times: user 146 ms, sys: 20.9 ms, total: 167 ms
Wall time: 1.81 s


In [75]:
df_engineer.cache()
df_engineer.show(5, truncate=False)

+-------------------+-------------------------------------------------------------------------------------------------------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------

In [76]:
df_cleaned.unpersist()

DataFrame[id: string, text: string, sarcastic: string]

# Model training

## Simple Feed Forward Network

### Word Embedding

In [ ]:
embeddings = WordEmbeddings.pretrained("glove_6B_300", "xx") \
      .setInputCols("sentence", "token") \
      .setOutputCol("embeddings")